# Longseq example

- Longseq introduction
- Making Sequence datasets from long sentence
- Adding FC layer and stacking RNN
- Code run through

### Longseq introduction
쓸모있는 문제를 해결하기 위해선 긴 문장을 해결 할 수 있어야 합니다.

### __making sequence dataset from long sentence__   
긴 문장에서 윈도우를 한칸씩 옮기면서 x 와 y로 설정합니다. 


0 if you wan -> f you want  
1 f you want ->  you want   
2  you want  -> you want t  
3 you want t -> ou want to  
4 ou want to -> u want to   
5 u want to  ->  want to b  
6  want to b -> want to bu  
7 want to bu -> ant to bui   
8 ant to bui -> nt to buil  
9 nt to buil -> t to build  
10 t to build ->  to build   

X -> Y

### Addding FC Layer and stacking RNN
이전에는 순수 RNN만 가지고 있는 모델 Vanila RNN 모델을 살펴보았습니다. 
하지만 모델이 데이터에 언더피팅이 된다면 모델을 좀더 크게 만들고 싶을수 있습니다.   
그래서 이번시간에는 RNN을 Stacking 한후 FC layer를 통과하는 모델을 만들어 보도록 하겠습니다, 

In [2]:
import torch
import torch.optim as optim
import numpy as np

In [3]:
# Random seed to make results deterministic and reproducible
torch.manual_seed(777)

In [4]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

In [7]:
# make dictionary
char_set = list(set(sentence))
char_dic = {c : i for i,c in enumerate(char_set)}

In [15]:
# hyper parameters
dic_size = len(char_dic)
hidden_size = len(char_dic)
sequence_length = 10  # Any arbitrary number
learning_rate = 0.1

In [16]:
# data setting
x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1]
    print(i, x_str, '->', y_str)

    x_data.append([char_dic[c] for c in x_str])  # x str to index
    y_data.append([char_dic[c] for c in y_str])  # y str to index

x_one_hot = [np.eye(dic_size)[x] for x in x_data]

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

In [17]:
# transform as torch tensor variable
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [18]:
# declare 2RNN + FC
class Net(torch.nn.Module) :
    def __init__(self, input_dim, hidden_dim, layers) :
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        # number_layer 을 통해 몇층을 쌓을지 결정한다.
        self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)
    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

net = Net(dic_size, hidden_size, 2)

In [19]:
# loss & optimizer setting
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [20]:
# start training
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        # print(i, j, ''.join([char_set[t] for t in result]), loss.item())
        if j == 0:
            predict_str += ''.join([char_set[t] for t in result])
        else:
            predict_str += char_set[result[-1]]

    print(predict_str)

rrrrrrrrrrrrrrrrrrlrlrrrrrrrrrrrrrrrlrlrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrlrrrrrlrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrlrrrrrrrrrrrrrrrrrrrrrrrrrrlrrrrrlrrrrrrrrrrrr
                                                                                                                                               o                                   
                                                                                                                                                                                   
oaa t taet taa tt t s ta t tat  taat t t t ta t .a s . et't t ot .a.a tt tt t saa  aat t t t t  tat .t o  taa  tat  taa  t t t ta . .tt t .t .a.t sata t t ot yt.t ao  ta  i a ta.t
ttiat tat itos tasi t t t tlt t tlt t tat itot ta it t taitai tatlta tas iati t t tlt t tot ittttoi tat t tot tlt t t t tat at t t t taat tat tat tlt tat tat tat tat tat itatstat 
 rmon gtositos gtorotoroioratorototoitoto rtoi rt ge retsitos gt sts roo gtos tos gotoiittsioeortosi

p rou want to build a ship, don't drum up people together te collect wood and don't dssign the  tosks and work, but rather teach them to long for the tndless immensity of the seac
p rou want to build a ship, don't arum up people together to collect wood and don't assign ther tosks and work, but rather teach them to longefor themendless immensity of the eet.
t iou want to build a ship, don't arum up people together te collect wood and don't assign them tosks and work, but rather teach them to long for the endless immensity of the eea.
t rou want to luild a ship, don't arum up people together to collect wood and don't assign them tosks and work, but rather teach them to long for the endless immensity of the s a.
t iou want to build a ship, don't arum up people together te collect wood and don't assign them tosks and work, but rather teach them to long for the endless immensity of the set.
t rou want to build a ship, don't arum up people together to collect wood and don't assign them tosk

m you want to build a ship, don't drum up people together to collect wood and don't assign them tosks and work, but rather teach them to long for the endless immensity of the sea.
m you want to build a ship, don't drum up people together to collect wood and don't assign them tosks and work, but rather teach them to long for the endless immensity of the sea.
m you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
t you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
f you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
f you want to build a ship, don't drum up people together to collect wood and don't assign them task

__정리__  
어떻게 하면 데이터셋을 균일한 사이즈로 잘라서 준비할지,  
그리고 파이토치에서 vanila RNN에  추가적으로 stacking 이나 FC등을 연결하는방법 